In [12]:
import os,os.path
import csv
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize

#  Create documents

In [ ]:
# creating a .tsv document for each review
with open('Airbnb_Texas_Rentals.tsv', 'r', encoding = "utf8") as all_rev:
    csv_reader = csv.reader(all_rev)
    ind = -1
    for row in all_rev:
        ind += 1
        if ind == 0:
            continue
        # skipping the empty lines
        else if row == '\n':
            ind -= 1
            # we manipulate 'ind' in order to have files named doc_i where i = 1,2,3,..
            else:
                # we store the documents inside a new folder
                new_title = r'fileAIR\doc_' + str(ind) + '.tsv'
                csv_writer = csv.writer(open(new_title, 'w', encoding = "utf8"), delimiter ='\t')
                # we skip the first column which is 
                row1 = row.split("\t")[1:]
                csv_writer.writerow(row1)

# Search Engine

In [22]:
# function to preprocess the files
def preprocess(something):
    something = something.lower()
    # removing '\n'
    something = something.replace('\\n', ' ')
    # removing punctuation
    tokenizer = regexp_tokenize(something, "[\w'\$]+")
    # filter the non stopwords
    filtered = [w for w in tokenizer if not w in stopwords.words('english')]
    ps = PorterStemmer()
    # removing the stem
    filtered = [ps.stem(word) for word in filtered]
    return filtered

In [13]:
len_file = len([x for x in os.scandir(r'C:\Users\alice\Desktop\ADM_HW3\fileAIR')])
print (len_file)

264


In [23]:
vocabulary_set = set()
docs_list = []

In [24]:
for i in range(1, len_file):
    with open(r'C:\Users\alice\Desktop\ADM_HW3\fileAIR\doc_' + str(i) + '.tsv', 'r', encoding = 'utf8') as csvfile:
        file1 = csv.reader(csvfile, delimiter = '\t')
        columns = [i for i in file1]
        description = columns[0][4]
        description = preprocess(description)
        title = columns[0][7]
        title = preprocess(title)
        tit_desc = title + description
        vocabulary_set.update(tit_desc)
        docs_list.append(set(tit_desc))

In [25]:
vocabulary = {k:v for v, k in enumerate(vocabulary_set)}
vocabulary

{'premium': 0,
 'pedernal': 1,
 'blanco': 2,
 'prime': 3,
 'quaint': 4,
 'counti': 5,
 'advantag': 6,
 'cargo': 7,
 'lap': 8,
 'pick': 9,
 '69': 10,
 'kyle': 11,
 'standard': 12,
 'flag': 13,
 'coffeepot': 14,
 'ninja': 15,
 'gourmet': 16,
 'deep': 17,
 'season': 18,
 'hospit': 19,
 'cabin': 20,
 'overlook': 21,
 'anybodi': 22,
 'among': 23,
 'fool': 24,
 'sq': 25,
 'highway': 26,
 'nine': 27,
 'glass': 28,
 'hood': 29,
 '118': 30,
 'chao': 31,
 'stay': 32,
 'westmoreland': 33,
 'countri': 34,
 'six': 35,
 'beverag': 36,
 '7th': 37,
 'metal': 38,
 'guesthous': 39,
 'techo': 40,
 'marriott': 41,
 'tempurped': 42,
 'seen': 43,
 'awar': 44,
 'trampolin': 45,
 "chic'": 46,
 'shadi': 47,
 'adventur': 48,
 'mile': 49,
 'piper': 50,
 'sqft': 51,
 'wonder': 52,
 'more': 53,
 'del': 54,
 'townhom': 55,
 'builder': 56,
 'secur': 57,
 '1ba': 58,
 'mind': 59,
 'wbc': 60,
 'rm': 61,
 'ideal': 62,
 'neighboorhood': 63,
 'tax': 64,
 'control': 65,
 'real': 66,
 'bad': 67,
 "tummie'": 68,
 "int'l": 69

In [26]:
# save vocabulary to the file
voc_file = open(r'C:\Users\alice\Desktop\ADM_HW3\vocabulary.txt', 'w', encoding = "utf8")
for term in vocabulary:
    voc_file.write('{0}\t{1}\n'.format(term, vocabulary[term]))

In [33]:
# we use defaultdict because it gives no key error
from collections import defaultdict 
#we need t sort this value
inv_indx = defaultdict(set)
for idx, text in enumerate(docs_list):
    for word in text:
        id_word = vocabulary[word]
        inv_indx[id_word].add(idx)

In [85]:
# to have a list as the value inside the dict
for key in inv_indx:
    value = inv_indx[key]
    inv_indx[key] = list(value)

inv_indx

In [35]:
# save inv_indx to the .txt file
inv_file = open(r'C:\Users\alice\Desktop\ADM_HW3\inverted_indx.txt', 'w', encoding = "utf8")
for id_term in inv_indx:
    docks = inv_indx[id_term]
    d = '\t'.join(map(str, docks))
    inv_file.write('{0}\t{1}\n'.format(id_term, d))

In [74]:
query = str(input())

private room bathroom


In [92]:
def SearchEngine(query, vocab, inv_indx):
    doc_list = []
    li = preprocess(query)
    pos_list = []
    for item in li:
        if item not in vocabulary:
            print('No documents found')
            break
        pos_id = vocabulary[item]
        pos_list.append(pos_id)
    result_list = []
    for term_id in pos_list:
        result_list.append(inv_indx[term_id])
    return result_list

In [115]:
def findTheSomething(res_list):
    test = {k:1 for k in res_list[0]}
    for sublist_ind in range(1, len(res_list)):
        for k in res_list[sublist_ind]:
            try:
                test[k] += 1
            except:
                test[k] = 1
    return [doc_id for doc_id in test if test[doc_id]==3]

In [94]:
pos = SearchEngine(query, vocabulary, inv_indx)
pos

[[0,
  256,
  3,
  131,
  132,
  259,
  7,
  9,
  152,
  26,
  27,
  159,
  160,
  34,
  36,
  39,
  169,
  170,
  172,
  45,
  174,
  175,
  178,
  179,
  181,
  55,
  56,
  184,
  58,
  185,
  60,
  64,
  65,
  66,
  67,
  192,
  69,
  194,
  71,
  200,
  201,
  74,
  75,
  76,
  80,
  208,
  83,
  84,
  85,
  212,
  216,
  217,
  90,
  92,
  220,
  105,
  106,
  233,
  108,
  237,
  110,
  240,
  114,
  243,
  254,
  250,
  251,
  252,
  126],
 [0,
  1,
  2,
  3,
  130,
  131,
  258,
  7,
  137,
  138,
  11,
  140,
  13,
  20,
  21,
  148,
  151,
  152,
  26,
  27,
  31,
  159,
  33,
  36,
  39,
  168,
  169,
  43,
  171,
  45,
  172,
  173,
  175,
  178,
  52,
  53,
  183,
  56,
  57,
  184,
  185,
  60,
  62,
  63,
  64,
  67,
  69,
  71,
  201,
  76,
  77,
  206,
  80,
  209,
  83,
  211,
  86,
  88,
  91,
  219,
  220,
  94,
  95,
  221,
  97,
  224,
  100,
  104,
  105,
  233,
  110,
  239,
  241,
  114,
  115,
  116,
  118,
  119,
  246,
  249,
  250,
  127],
 [0,
  1,
  3,
  

In [117]:
doc = findTheSomething(pos)
doc

[0, 3, 7, 152, 26, 159, 39, 175, 185, 64, 220]